# Data Collection from APIs

In [1]:
import asyncio
import aiohttp
import json
import pandas as pd

## Collecting Data From Four Data Sources
- apps_json() collects data from steam's official api where we get App name and id
- steamspy_json() collects data from steam and is a third party steam observer where we get multiple metrics and info such as ratings, reviews, etc.
- steamhf_json() collects data from steam and is a third party steam observer where it is very similar to steamspy but different in that it brings in different info such as genres, categories, and tags
- gamalytic_json() is another 3rd party steam observer which is gets two metrics, the class of the publisher which entails how high tier a game it is and the amount of copies sold of the game

In [2]:
async def fetch(session, url):
    try:
        async with session.get(url) as response:
            if response.status == 200:
                return await response.text()
            else:
                print(f"Failed to fetch {url}: Status {response.status}")
                return None
    except Exception as e:
        print(f"Exception occurred while fetching {url}: {e}")
        return None

async def apps_json():
    game_id_url = "https://api.steampowered.com/ISteamApps/GetAppList/v2/"
    async with aiohttp.ClientSession() as session:
        response_id = await fetch(session, game_id_url)
        if response_id:
            game_dict = json.loads(response_id)['applist']['apps']
            return game_dict
        return {}

async def steamspy_json():
    steamspy_dict = dict()
    index = 0
    page = 0
    steamspy_columns = ['appid', 'positive', 'negative', 
                        'userscore', 'score_rank', 'price', 
                        'owners', 'ccu', 'developer', 'publisher']
    
    async with aiohttp.ClientSession() as session:
        while True:
            steamspy_url = f"https://steamspy.com/api.php?request=all&page={page}"
            steamspy_response_text = await fetch(session, steamspy_url)
            if steamspy_response_text:
                steamspy_json = json.loads(steamspy_response_text)
                if not steamspy_json:
                    break
                for i in steamspy_json:
                    app = steamspy_json[i]
                    info = {element: str(app[element]) for element in steamspy_columns}
                    steamspy_dict[index] = info
                    index += 1
                page += 1
            else:
                break
        
    return steamspy_dict

async def steamhf_json():
    steamhf_columns = ['AppID', 'Estimated owners', 'Price', 
                    'Metacritic score', 'User score', 'Positive', 'Negative', 
                    'Recommendations', 'Categories', 'Tags', 'Peak CCU', 'Genres' ]
    steamhf_dict = dict()
    offset = 0
    index = 0
    
    async with aiohttp.ClientSession() as session:
        while True:
            steamhf_url = f"https://datasets-server.huggingface.co/rows?dataset=FronkonGames%2Fsteam-games-dataset&config=default&split=train&offset={offset}&length=100"
            steamhf_response_text = await fetch(session, steamhf_url)
            if steamhf_response_text:
                steamhf_json = json.loads(steamhf_response_text)
                rows = steamhf_json.get('rows', [])
                if not rows:
                    break
                for i in range(len(rows)):
                    row_info = {column.lower(): rows[i]['row'][column] for column in steamhf_columns}
                    steamhf_dict[index] = row_info
                    index += 1
                offset += 100
            else:
                break
        
    return steamhf_dict

async def gamalytic_json():
    gamalytic_cols = ['steamId', 'publisherClass', 'copiesSold']
    gamalytic_dict = dict()
    page = 0
    index = 0
    
    async with aiohttp.ClientSession() as session:
        while True:
            gamalytic_url = f"https://api.gamalytic.com/steam-games/list?page={page}&fields=copiesSold,publisherClass,steamId"
            gamalytic_response_text = await fetch(session, gamalytic_url)
            if gamalytic_response_text:
                gamalytic_json = json.loads(gamalytic_response_text)
                results = gamalytic_json.get('result', [])
                if not results:
                    break
                gamalytic_data = [{col.lower(): result.get(col, None) for col in gamalytic_cols} for result in results]
                for row in gamalytic_data:
                    gamalytic_dict[index] = row
                    index += 1
                page += 1
            else:
                break
        
    return gamalytic_dict


## General View of each DataFrame

In [3]:
apps_df = pd.DataFrame(await apps_json())
apps_df['appid'] = apps_df['appid'].astype(int)
print(apps_df.shape)
apps_df.sample(n=5)

(202864, 2)


,appid,name
187553,1624080,sfäre Demo
148482,1022610,Alchemic DungeonsDX
58680,1879930,Naiad Demo
178156,2410130,GateTail
83577,1315280,PositronX Demo


In [4]:
steamhf_df = pd.DataFrame((await steamhf_json()).values())
steamhf_df['appid'] = steamhf_df['appid'].astype(int)
print(steamhf_df.shape)
steamhf_df.sample(n=5)

(83560, 12)


,appid,estimated owners,price,metacritic score,user score,positive,negative,recommendations,categories,tags,peak ccu,genres
32581,1164480,0 - 20000,14.99,0,0,10,2,0,"Single-player,Steam Achievements,Full controll...","RPG,RPGMaker,JRPG,Dating Sim,Farming Sim,LGBTQ...",0,RPG
63903,1966500,0 - 0,0.00,0,0,0,0,0,"Multi-player,PvP,Online PvP,Co-op,Online Co-op...",None,0,"Free to Play,Strategy"
2170,445350,0 - 20000,14.99,0,0,7,10,0,"Single-player,Multi-player,PvP,Online PvP,Co-o...","Strategy,Action,Early Access,Simulation,Co-op,...",0,"Action,Simulation,Strategy,Early Access"
172,763350,0 - 20000,2.99,0,0,8,1,0,"Single-player,Multi-player,PvP,Shared/Split Sc...","Casual,Action,Sports,Indie",0,"Action,Casual,Indie,Sports"
6580,562310,0 - 20000,2.99,0,0,11,1,0,"Single-player,Steam Achievements,Full controll...","Action,Indie,Casual",0,"Action,Casual,Indie"


In [7]:
steamspy_df = pd.DataFrame((await steamspy_json()).values())
steamspy_df['appid'] = steamspy_df['appid'].astype(int)
print(steamspy_df.shape)
steamspy_df.sample(n=5)

Failed to fetch https://steamspy.com/api.php?request=all&page=74: Status 500
(73467, 10)


,appid,positive,negative,userscore,score_rank,price,owners,ccu,developer,publisher
1394,1328660,7455,1829,0,,2999,"500,000 .. 1,000,000",192,Stellar Entertainment Limited,Electronic Arts
55162,1289170,7,0,0,,999,"0 .. 20,000",0,Jura Ex Alto,Jura Ex Alto
51583,298420,12,6,0,,599,"0 .. 20,000",0,Etter Studio,Etter Studio
5413,1282150,2317,171,0,,3999,"100,000 .. 200,000",25,Purple Lamp,THQ Nordic
63652,839940,25,1,0,,799,"0 .. 20,000",1,Choice of Games,Choice of Games


In [8]:
gamalytic_df = pd.DataFrame((await gamalytic_json()).values())
gamalytic_df = gamalytic_df.rename(columns={'steamid': 'appid'})
gamalytic_df['appid'] = gamalytic_df['appid'].astype(int)
print(gamalytic_df.shape)
gamalytic_df.sample(n=5)

(82277, 3)


,appid,publisherclass,copiessold
68278,1867490,Hobbyist,1
80293,2528860,Indie,7311
51350,1792770,Hobbyist,25
4360,1249040,AA,38632
75011,1069220,Hobbyist,1046


## Merging All Dataframes Into Primary Dataset

In [11]:
df = pd.merge(steamspy_df, gamalytic_df, how='left', on='appid')
print(df.shape)
df.sample(n=5)

(73467, 12)


,appid,positive,negative,userscore,score_rank,price,owners,ccu,developer,publisher,publisherclass,copiessold
69961,2789380,1,0,0,,399,"0 .. 20,000",0,Deez Games,Deez Games,Hobbyist,15.0
51024,1065050,1,2,0,,199,"0 .. 20,000",0,Sonic-Alpha,NS,Hobbyist,54.0
43946,1051320,1,2,0,,299,"0 .. 20,000",0,IO Games,IO Games,Indie,36.0
18169,1756120,34,63,0,,99,"20,000 .. 50,000",0,Boom Games,Boom Games,Indie,2822.0
22070,1755300,136,54,0,,149,"20,000 .. 50,000",3,Turquoise Revival Games,"GrabTheGames, IndieArk",Indie,6509.0


In [13]:
df = pd.merge(df, steamhf_df, how='left', on='appid')
print(df.shape)
df.sample(n=5)

(73467, 34)


,appid,positive_x,negative_x,userscore,score_rank,price_x,owners,ccu,developer,publisher,...,price,metacritic score_y,user score_y,positive,negative,recommendations_y,categories_y,tags_y,peak ccu_y,genres_y
58334,1352870,3,2,0,,399,"0 .. 20,000",0,Droid Riot,Conglomerate 5,...,4.99,0.0,0.0,0.0,0.0,0.0,Single-player,None,0.0,Action
62473,2798810,5,2,0,,949,"0 .. 20,000",1,Denzee Games,Denzee Games,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19675,328440,43,53,0,,699,"20,000 .. 50,000",4,"Accolade, Inc.","Atari, Nightdive Studios",...,1.39,0.0,0.0,30.0,51.0,0.0,Single-player,"Strategy,Turn-Based,Sci-fi",0.0,Strategy
73221,1648760,46,3,0,,1999,"0 .. 20,000",1,sqr3lab,sqr3lab,...,16.99,0.0,0.0,0.0,0.0,0.0,"Single-player,Full controller support",None,2.0,"Adventure,Simulation,Early Access"
19860,604920,15,1,0,,999,"20,000 .. 50,000",0,Omaj,Blossomsoft,...,9.99,0.0,0.0,14.0,1.0,0.0,"Single-player,Steam Achievements,Full controll...","RPG,JRPG,Singleplayer,Pixel Graphics,Drama,RPG...",0.0,RPG


In [14]:
df = pd.merge(df, apps_df, how='left', on='appid')
print(df.shape)
df.sample(n=5)

(73467, 35)


,appid,positive_x,negative_x,userscore,score_rank,price_x,owners,ccu,developer,publisher,...,metacritic score_y,user score_y,positive,negative,recommendations_y,categories_y,tags_y,peak ccu_y,genres_y,name
56695,1238720,5,1,0,,99,"0 .. 20,000",0,EmotionalMingo,EmotionalMingo,...,0.0,0.0,4.0,1.0,0.0,Single-player,"Indie,Adventure,Platformer,Shooter,Beautiful,F...",0.0,"Adventure,Indie",Wandering Through The Vines
41961,2433910,1,1,0,,1499,"0 .. 20,000",0,Brothers Grimm Studios,Brothers Grimm Studios,...,0.0,0.0,0.0,0.0,0.0,"Multi-player,Co-op,Online Co-op",None,0.0,"Action,Indie,Early Access",Escape The Office
41784,1932260,163,22,0,,1999,"0 .. 20,000",6,MinMax Games Ltd.,MinMax Games Ltd.,...,0.0,0.0,52.0,9.0,0.0,"Single-player,Multi-player,Co-op,Online Co-op,...","Early Access,Tower Defense,Strategy,Singleplay...",25.0,"Strategy,Early Access",No Creeps Were Harmed TD
13179,1218170,357,133,0,,699,"20,000 .. 50,000",2,漫森游戏工作室,漫森游戏工作室,...,0.0,0.0,261.0,88.0,292.0,Single-player,"Horror,Psychological Horror,Adventure,Historic...",7.0,"Action,Adventure,Casual,Indie,RPG,Simulation",回门 Way Back Home
25699,362920,33,62,0,,399,"0 .. 20,000",0,"Cryo Interactive, Koalabs",Microids,...,0.0,0.0,25.0,55.0,0.0,Single-player,"Adventure,Point & Click",1.0,Adventure,Atlantis 2: Beyond Atlantis


In [23]:
columns = ['appid', 'name', 'positive_x', 'negative_x',
           'price_y', 'copiessold', 'metacritic score_x', 'recommendations_x',
           'ccu', 'peak ccu_x', 'developer', 'publisher', 'publisherclass',
            'categories_x', 'tags_x', 'genres_x']
columns_formatted = ['appid', 'name', 'positive', 'negative',
           'price', 'copies_sold', 'metacritic_score', 'recommendations',
           'current_users', 'peak_users', 'developer', 'publisher', 'publisherclass',
            'categories', 'tags', 'genres']

df = df[columns]
df = df.rename(columns={columns[i]:columns_formatted[i] for i in range(len(columns))})

In [26]:
df.sample(n=5)

,appid,name,positive,negative,price,copies_sold,metacritic_score,recommendations,current_users,peak_users,developer,publisher,publisherclass,categories,tags,genres
3702,497640,Trick & Treat,2186,85,0.00,196891.0,0.0,0.0,2,3.0,Rabbiton,Rabbiton,Indie,"Single-player,Steam Achievements,Steam Cloud","Free to Play,RPGMaker,Pixel Graphics,Adventure...","Adventure,Casual,Free to Play,Indie"
72437,972910,Timore 6,31,7,9.99,1365.0,0.0,0.0,0,0.0,Vidas Salavejus,Vidas Salavejus,Indie,Single-player,"Indie,Gore,Violent,Nudity,Horror,Psychological...",Indie
11723,347510,NaN,246,39,0.00,NaN,0.0,132.0,0,0.0,Citeremis Inc.,Citeremis Inc.,NaN,"Single-player,Steam Achievements,Full controll...","Indie,Action,Casual,Arcade,Stealth","Action,Casual,Indie"
7681,1889650,Jigsaw Bestiary,32,1,1.99,775.0,0.0,0.0,2,1.0,Minicactus Games,Minicactus Games,Indie,"Single-player,Steam Achievements,Steam Cloud","Puzzle,Tabletop,Hand-drawn,Mythology,Historica...",Casual
21845,2103660,Bugz Bows and Curses,1,0,NaN,1.0,NaN,NaN,0,NaN,Prestosilver,Prestosilver,Hobbyist,NaN,NaN,NaN


In [27]:
df.to_excel('steam_sales.xlsx', index=False)